In [1]:
%env AWS_REQUEST_PAYER=requester

env: AWS_REQUEST_PAYER=requester


In [2]:
from stacLib.ard_stac_search import Astac
import rasterio as rio
from rasterio.windows import Window

import numpy as np
import tqdm.notebook as tq
import zarr

In [3]:
work = 'H21V10_Y18X7'

In [4]:
def get_work(work_str):
    
    next = work_str.split('X')[0]
    X = work_str.split('X')[1]
    #print(next, X)
    Y = next.split('Y')[1]
    nex = next.split('Y')[0]
    ne, V_ = nex.split('V')
    V = V_.replace('_','')
    H = ne.replace('H','')
    #print(H,V,Y,X)
    return (H,V,Y,X)
    

In [5]:
my_date='2020-01-01/2024-12-31'

In [6]:
stac_config = {}

stac_config['url'] = 'https://landsatlook.usgs.gov/stac-server'
stac_config['fav_collection'] = 'landsat-c2l3-dswe'
stac_config['description'] = 'Landsat Collection 2 ARD Tiles DSWE'


In [7]:
print(work)
(H,V,Y,X) = get_work(work)

H21V10_Y18X7


In [8]:
(H,V,Y,X)

('21', '10', '18', '7')

In [9]:
chunkY = 250
chunkX = 250

In [10]:
_stac = Astac(stac_config)

https://landsatlook.usgs.gov/stac-server


In [11]:
date_range_text = my_date
cloud_cover_pct_max = '100'
_assets = _stac.get_my_assets(None, H, V, date_range_text, cloud_cover_pct_max)

None 2020-01-01/2024-12-31 100


/wsefs/mambaforge/envs/water/lib/python3.11/site-packages/pystac_client/item_search.py:865: FutureWarning: get_all_items_as_dict() is deprecated, use item_collection_as_dict() instead.
  warnings.warn(


440 items found


In [12]:
_assets[0]['properties']

{'start_datetime': '2023-10-26T16:36:06.7757460Z',
 'end_datetime': '2023-10-26T16:36:30.6582900Z',
 'datetime': '2023-10-26T16:36:18.717018Z',
 'platform': 'LANDSAT_9',
 'instruments': ['OLI', 'TIRS'],
 'landsat:grid_horizontal': '21',
 'landsat:grid_vertical': '10',
 'landsat:grid_region': 'CU',
 'landsat:scene_count': 2,
 'eo:cloud_cover': 93.1232,
 'landsat:cloud_shadow_cover': 5.4688,
 'landsat:snow_ice_cover': 0.0006,
 'landsat:fill': 45.423,
 'proj:epsg': None,
 'proj:shape': [5000, 5000],
 'proj:transform': [30, 0, 584415, 0, -30, 1814805],
 'created': '2023-10-30T14:11:01.126Z',
 'updated': '2023-10-30T14:11:01.126Z'}

In [13]:

def read_tif(path: str, band: int = 1, window: Window = None):
    return rio.open(path).read(band, window=window)

def read_ls_data(item, band, window):
    return read_tif(item['assets'][band]['alternate']['s3']['href'], window=window)

In [14]:
len(_assets)

440

In [15]:
# Initialize 3d array time Y X

In [16]:
window = Window(0, 0, chunkY, chunkX)

In [17]:
big3d_ary = np.zeros(shape=(len(_assets), chunkY, chunkX), dtype=np.uint16)

In [18]:
big3d_ary.shape

(440, 250, 250)

In [19]:
%%time
p_bar=tq.tqdm(range(len(_assets)), position=0, leave=True)
myband='intsm'
for t1 in range(len(_assets)):
    item = _assets[t1]
    big3d_ary[t1] = read_ls_data(item, myband, window)
    p_bar.update(1)
    p_bar.refresh()

  0%|          | 0/440 [00:00<?, ?it/s]

CPU times: user 11.4 s, sys: 801 ms, total: 12.2 s
Wall time: 2min 1s


In [20]:
Z_store = f's3://ws-out/dwse/_H{H}V{V}_Y{Y}X{X}_store.zarr'

In [21]:
%%time
# Save the 3D NumPy array as a Zarr array
zarr.save(Z_store, big3d_ary)

CPU times: user 1 s, sys: 320 ms, total: 1.32 s
Wall time: 5.94 s
